 ### Solving Capacitated Lot Sizing Problem using CPLEX's python module DOCPLEX

In [119]:
import random as r
import numpy as np

r.seed(50)

no_of_items = 10
no_of_machines = 2
no_of_period = 5

# setting up the cost
low_setup_time = np.asarray([r.randint(a=10,b=50) for i in range(no_of_items*no_of_machines)])
high_setup_time = 1.5*low_setup_time

low_setup_cost = np.asarray([r.randint(a=5,b=95) for i in range(no_of_items*no_of_machines)])
high_setup_cost = 10*low_setup_cost

unit_production_cost = [round(r.uniform(a=1.5,b=2.5), 2) for i in range(no_of_items*no_of_machines)]
unit_inventory_cost = [round(r.uniform(a=0.2,b=0.4),2) for i in range(no_of_items*no_of_period)]
unit_production_time = [r.randint(a=1,b=5) for i in range(no_of_items*no_of_machines)]

demand = [round(r.uniform(a=0,b=180)) for i in range(no_of_items*no_of_period)]

In [120]:
# function to calculate average demand of each item per period
demand_per_period = []
def capacity(demand): 
    demand_per_period.clear()
    total_demand_per_period = 0
    for t in range(1,no_of_period+1):
        for i in range(1,no_of_items+1):
            total_demand_per_period += demand[i,t]
            average_per_period = total_demand_per_period/10
        demand_per_period.append(average_per_period)
    return demand_per_period

In [121]:
# Assigning Costs
Item_list = [i for i in range(1,no_of_items+1)]
Machine_list = [j for j in range(1,no_of_machines+1)]
Period_list = [t for t in range(1,no_of_period+1)]

item_machine_pair = [(i,j) for i in Item_list for j in Machine_list]
item_period_pair = [(i,t) for i in Item_list for t in Period_list]
machine_period_pair = [(j,t) for j in Machine_list for t in Period_list]

Bij = {(i,j): s for (i,j),s in zip(item_machine_pair,unit_production_time)}   # Unit production time of item i on machine j;
Cij = {(i,j): s for (i,j),s in zip(item_machine_pair,unit_production_cost)}   # Unit production cost of item i on machine j;
Hit = {(i,t): s for (i,t),s in zip(item_period_pair,unit_inventory_cost)}     # Unit inventory cost of item i per period t;
Dit = {(i,t): s for (i,t),s in zip(item_period_pair,demand)}                  # Demand of item i at period t;
Fij = {(i,j): s for (i,j),s in zip(item_machine_pair,low_setup_time)}         # Setup time of item i on machine j;
Sij = {(i,j): s for (i,j),s in zip(item_machine_pair,low_setup_cost)}         # Setup cost of item i on machine j;
capacity(Dit)
Qjt = {(j,t): s for (j,t),s in zip(machine_period_pair,demand_per_period*2)}  # Capacity of machine j at period t;
Mt = {t: s for t,s in zip(Period_list,demand_per_period)}                     # Maximum capacity of machines per period.

In [122]:
# print("Bij: {}\n\nCij: {}\n\nHi: {}\n\nDit: {}\n\nFij: {}\n\nSij: {}\n\nQjt: {}".format(Bij,Cij,Hi,Dit,Fij,Sij,Qjt))
# print(Mt)

In [123]:
# Defining variables
Vars = []
for items in Item_list:
    for machines in Machine_list:
        for periods in Period_list:
            Vars.append((items,machines,periods))
            
Inventory_vars = []
for items in Item_list:
    for periods in Period_list:
        Inventory_vars.append((items,periods))

In [124]:
from docplex.mp.model import Model

In [125]:
# Initializing the Model
mip = Model('CLSP')

In [126]:
# Decision variables
X = mip.integer_var_dict(Vars, lb=0, name='x')
Y = mip.binary_var_dict(Vars,name='y')
S = mip.integer_var_dict(Inventory_vars, lb=0, name='s')

In [127]:
# Constraints
constraint_1 = mip.add_constraints(mip.sum(X[i,j,t] for j in range(1,no_of_machines+1)) + (-S[i,t] + S[i,t-1] if t-1 > 0 else -S[i,t]) == Dit[i,t] for i,t in S)
constraint_2 = mip.add_constraints(mip.sum(Bij[i,j]*X[i,j,t] + Fij[i,j]*Y[i,j,t] for i in range(1,no_of_items+1)) <= Qjt[j,t] for j,t in Qjt)
constraint_3 = mip.add_indicator_constraints(mip.indicator_constraint(Y[i,j,t], X[i,j,t] <= Mt[t]) for i,j,t in X)

In [128]:
# Objective function
setup_cost = mip.sum(Sij[i,j]*Y[i,j,t] for i,j,t in Y) 
production_cost = mip.sum(Cij[i,j]*X[i,j,t] for i,j,t in X)
inventory_cost = mip.sum(Hit[i,t]*S[i,t] for i,t in S)
total_cost = setup_cost + production_cost + inventory_cost

In [129]:
# Goal
mip.minimize(total_cost)

In [130]:
mip.print_information()

Model: CLSP
 - number of variables: 250
   - binary=100, integer=150, continuous=0
 - number of constraints: 160
   - linear=60, indicator=100
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [ ]:
solution = mip.solve(log_output=True)
assert solution is not None
value = solution.objective_value

In [ ]:
# print(solution.print_mst())

In [ ]:
# solution.solve_status